In [1]:
%%html
<style> .container { width: 100% !important; }</style>

In [2]:

import pandas as pd
pd.set_option('max_colwidth', 200)  # Set a very large value
pd.set_option('max_rows', 2000)  # Set a very large value

def add_borders(val):
    return 'border: 1px solid black;'

style_df = lambda df : df.style.applymap(add_borders)

locations_df = pd.read_json("./LocationsParsed.json")
Channelslocations_df = pd.read_json("./Channel2location.json")

In [32]:
# drop all dbs
Channelslocations_df = Channelslocations_df[Channelslocations_df.chn_name.apply(lambda x:x.split('_')[0]!='d')].describe()

In [3]:
display (pd.DataFrame (locations_df.columns, columns=["columns"]))
display  (pd.DataFrame (Channelslocations_df.columns, columns=["columns"]))

columns
0           database
1        db_instance
2       db_node_name
3             dbdesc
4               host
5          loc_class
6        loc_db_user
7    loc_description
8           loc_name
9   loc_remote_login
10   loc_remote_node
11   loc_remote_port

columns
0        chn_name
1        act_name
2  act_parameters
3       loc_class
4        loc_name
5            host
6        database
7          dbdesc

In [4]:
locations_df.head(1).T

0
database                                                                         ushvr12
db_instance                                                         usinnovationredshift
db_node_name        us-innovation-redshift.c8ziwm1qxh67.us-east-1.redshift.amazonaws.com
dbdesc                                                                               dev
host              odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com
loc_class                                                                       redshift
loc_db_user                                                                    504005930
loc_description                                                                    tredp
loc_name                                                                           tredp
loc_remote_login                                                                 isc_dev
loc_remote_node                                                           10.242.112.153
loc_remote_port                                                                     8443

In [5]:
Channelslocations_df.head(1).T

0
chn_name                                                                       edhr_01
act_name                                                                       Capture
act_parameters                                                             /NoTruncate
loc_class                                                                       oracle
loc_name                                                                         sedhp
host            odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com
database                                                                       ushvr12
dbdesc                                                                             dev

In [6]:
locations_prod_df = locations_df[locations_df['dbdesc'].isin(['US hvr prod','fin-prod'])]

------------  Analysis

In [11]:
import pandas as pd

# Sample data
data = {
    'Category': ['A', 'B', 'A', 'B', 'A', 'B'],
    'Value1': [10, 20, 30, 40, 50, 60],
    'Value2': [100, 200, 300, 400, 500, 600]
}

df = pd.DataFrame(data)

# Define custom aggregation functions
def custom_agg_function1(series):
    return series.max() - series.min()

def custom_agg_function2(series):
    return series.sum()

# Group by 'Category' and apply different aggregation functions to different columns
result = df.groupby('Category').agg(
    RangeValue1=('Value1', custom_agg_function1),
    SumValue2=('Value2', custom_agg_function2)
)

print(result)


          RangeValue1  SumValue2
Category                        
A                  40        900
B                  40       1200


-- threre will be problem you can crate same location name with diffreent location class!!
-- we need to group in to single rows muliti source locaiton and multi target locations

In [12]:
Channelslocations_capture_df = Channelslocations_df[Channelslocations_df.act_name.apply(lambda x : x=='Capture')]
channel_capyture_df = pd.merge(Channelslocations_capture_df,locations_df  , on=['dbdesc','database','host','loc_name','host'], how='left') [['dbdesc','loc_class_y','db_node_name','database','chn_name','loc_remote_node','loc_name','loc_description']].sort_values(['dbdesc','loc_class_y','db_node_name','chn_name','database','loc_name']).fillna('')

channel_capyture_df.head()

dbdesc  loc_class_y    db_node_name database     chn_name  \
124  Finance Nprod  file_locdir  10.242.109.196  ushvr00    bsr_fr_01   
125  Finance Nprod  file_locdir  10.242.109.196  ushvr00    bsr_fr_02   
126  Finance Nprod  file_locdir  10.242.109.196  ushvr00    bsr_fr_03   
127  Finance Nprod  file_locdir  10.242.109.196  ushvr00    bsr_fr_04   
133  Finance Nprod  file_locdir  10.242.109.196  ushvr00  cpm_zip_tst   

    loc_remote_node loc_name loc_description  
124  10.242.109.196    sftpb  flat file prod  
125  10.242.109.196    sftpb  flat file prod  
126  10.242.109.196    sftpb  flat file prod  
127  10.242.109.196    sftpb  flat file prod  
133  10.242.109.196     szip  flat file prod

In [13]:
part_of= lambda x,y : (x in y or y in x ) if (x!='' and y!='')  else False
channel_capyture_df['node_eq_agent_f']=channel_capyture_df.apply(lambda x : part_of (x['db_node_name'],x['loc_remote_node']), axis=1)

channel_capyture_prod_df = channel_capyture_df[channel_capyture_df['dbdesc'].isin(['US hvr prod','fin-prod'])]


In [21]:
channel_capyture_prod_agg_df.columns

Index(['dbdesc', 'chn_name', 'database', 'cap_loc_classes', 'cap_loc_names',
       'cap_db_node_names', 'cap_loc_descriptions', 'cap_loc_remote_nodes',
       'cap_node_eq_agent_f'],
      dtype='object')

In [19]:
aggfunc = lambda x : "</br>".join(sorted(set(x)))

style_df(
channel_capyture_prod_agg_df.groupby(['dbdesc','cap_loc_classes','cap_db_node_names'])[['cap_loc_descriptions']].agg(aggfunc).sort_values(['dbdesc','cap_loc_classes'])
    )

In [18]:
channel_capyture_prod_agg_df.to_json('channel_capyture_prod_agg_df.json', orient='records', lines=True)

style_df(
channel_capyture_prod_agg_df.pivot_table(index = ['dbdesc','cap_loc_classes','cap_db_node_names'] , columns = 'database' , values = 'chn_name', aggfunc=len, margins=True, margins_name='Grand Total').fillna('').astype(str).apply(lambda x: x.str.rstrip('.0')).sort_values(['dbdesc','cap_loc_classes'])
    )

In [17]:
agg_func = lambda x : ' <br/> '.join(set (x))

cols  = ['dbdesc','cap_db_node_names','chn_name', 'database', 'cap_loc_names', 'cap_loc_descriptions', 'cap_loc_remote_nodes', 'cap_node_eq_agent_f']

channel_capyture_prod_agg_df = channel_capyture_prod_df.groupby(['dbdesc','chn_name','database']).agg(   cap_loc_classes      = ('loc_class_y'    ,agg_func) ,
                                                                                      cap_loc_names        = ('loc_name'    ,agg_func) ,
                                                                                      cap_db_node_names    = ('db_node_name',agg_func) ,
                                                                                      cap_loc_descriptions = ('loc_description',agg_func) ,
                                                                                      cap_loc_remote_nodes = ('loc_remote_node',agg_func) ,
                                                                                      cap_node_eq_agent_f  = ('node_eq_agent_f',set) ,
                                                                                ).reset_index()


channel_capyture_prod_agg_df.to_json('channel_capyture_prod_agg_df.json', orient='records', lines=True)

#channel_capyture_prod_agg_df = channel_capyture_prod_agg_df.sort_values(['dbdesc','cap_loc_classes','cap_db_node_names','chn_name'])

for cap_loc_classes in set (channel_capyture_prod_agg_df.cap_loc_classes) : 
    print (cap_loc_classes)
    display (style_df (
        channel_capyture_prod_agg_df[channel_capyture_prod_agg_df['cap_loc_classes']==cap_loc_classes][cols].set_index(['dbdesc','cap_db_node_names','chn_name','database']
                                                                                                                ).sort_values(['dbdesc','cap_db_node_names','chn_name'])
             )) 


teradata


oracle


file_s3s


postgresql


file_locdir <br/> file_s3s


...

teradata <br/> redshift


postgresql <br/> sqlserver


sqlserver


redshift


file_locdir


In [52]:
Channelslocations_df .groupby(['dbdesc','database','host','chn_name'])[['loc_class','act_name','loc_name']].agg(list).head(10)

loc_class  \
dbdesc        database host                                         chn_name                           
Finance Nprod ushvr00  odp-fin-nprod-hvr-metadata.odp.health.ge.com aligniq_fr_1  [oracle, redshift]   
                                                                    biosap_fr_02  [oracle, redshift]   
                                                                    biosap_fr_03  [oracle, redshift]   
                                                                    biosap_fr_04  [oracle, redshift]   
                                                                    biosap_fr_05  [oracle, redshift]   
                                                                    biosap_fr_06  [oracle, redshift]   
                                                                    biosap_fr_08  [oracle, redshift]   
                                                                    biosap_fr_09  [oracle, redshift]   
                                                                    biosap_fr_10  [oracle, redshift]   
                                                                    biosap_fr_11  [oracle, redshift]   

                                                                                              act_name  \
dbdesc        database host                                         chn_name                             
Finance Nprod ushvr00  odp-fin-nprod-hvr-metadata.odp.health.ge.com aligniq_fr_1  [Capture, Integrate]   
                                                                    biosap_fr_02  [Capture, Integrate]   
                                                                    biosap_fr_03  [Capture, Integrate]   
                                                                    biosap_fr_04  [Capture, Integrate]   
                                                                    biosap_fr_05  [Capture, Integrate]   
                                                                    biosap_fr_06  [Capture, Integrate]   
                                                                    biosap_fr_08  [Capture, Integrate]   
                                                                    biosap_fr_09  [Capture, Integrate]   
                                                                    biosap_fr_10  [Capture, Integrate]   
                                                                    biosap_fr_11  [Capture, Integrate]   

                                                                                        loc_name  
dbdesc        database host                                         chn_name                      
Finance Nprod ushvr00  odp-fin-nprod-hvr-metadata.odp.health.ge.com aligniq_fr_1   [aiqp, tredn]  
                                                                    biosap_fr_02  [ssapp, tredn]  
                                                                    biosap_fr_03  [ssapp, tredn]  
                                                                    biosap_fr_04  [ssapp, tredn]  
                                                                    biosap_fr_05  [ssapp, tredn]  
                                                                    biosap_fr_06  [ssapp, tredn]  
                                                                    biosap_fr_08  [ssapp, tredn]  
                                                                    biosap_fr_09  [ssapp, tredn]  
                                                                    biosap_fr_10  [ssapp, tredn]  
                                                                    biosap_fr_11  [ssapp, tredn]

In [17]:
part_of= lambda x,y : (x in y or y in x ) if (x!='' and y!='')  else False

part_of ("d","")

False

In [18]:
result = df.groupby('Category').agg(
    RangeValue1=('Value1', custom_agg_function1),
    SumValue2=('Value2', custom_agg_function2)
)

In [19]:
Channelslocations_df[Channelslocations_df.act_name.apply(lambda x : x=='Integrate')].groupby(['dbdesc','database','host','chn_name'])[['loc_class','act_name','loc_name']].agg(IntegrateClasses=('loc_class', list) ).head(200)

IntegrateClasses
dbdesc        database host                                                                 chn_name                                    
Finance Nprod ushvr00  odp-fin-nprod-hvr-metadata.odp.health.ge.com                         aligniq_fr_1                      [redshift]
                                                                                            biosap_fr_02                      [redshift]
                                                                                            biosap_fr_03                      [redshift]
                                                                                            biosap_fr_04                      [redshift]
                                                                                            biosap_fr_05                      [redshift]
                                                                                            biosap_fr_06                      [redshift]
                                                                                            biosap_fr_08                      [redshift]
                                                                                            biosap_fr_09                      [redshift]
                                                                                            biosap_fr_10                      [redshift]
                                                                                            biosap_fr_11                      [redshift]
                                                                                            biosap_fr_14                      [redshift]
                                                                                            biosap_fr_15                      [redshift]
                                                                                            biosap_fr_16                      [redshift]
                                                                                            biosap_kn                         [redshift]
                                                                                            biosap_mje_1                      [redshift]
                                                                                            bsr_fr_01                         [redshift]
                                                                                            bsr_fr_02                         [redshift]
                                                                                            bsr_fr_03                         [redshift]
                                                                                            bsr_fr_04                         [redshift]
                                                                                            cpm_fr_01                         [redshift]
                                                                                            cpm_fr_02                         [redshift]
                                                                                            cpm_fr_03                         [redshift]
                                                                                            cpm_fr_04                         [redshift]
                                                                                            cpm_fr_05                         [redshift]
                                                                                            cpm_zip_tst                       [redshift]
                                                                                            deman_fr_01                       [redshift]
                                                                                            drm_fr_01     [redshift, redshift, redshift]
                                                                                            drm_fr_02                         [redshift]
          